Study of Dark Current changes in SiPMs of STAR Event Plane Detector. 

Based on  Tristan Protzman code:  https://gist.github.com/tlprotzman/9c87f9f98d44a9be01385a90821560ed


In [ ]:


import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import glob
import os


In [2]:


# Directory containing the CSV files
directory = '/Users/stefaniak.9/OneDriveOSU/EPD/DarkCurrent_study/Ivscan/'
columns = ['timestamp', 'address', 'ew', 'pp', 'tt', 'fee_channel', 'fee', 'VCAL0', 'VPED', 'VSLOPE', 'VSET', 'VCOMP', 'IMON', 'RMON', 'TUFF', 'TUFF_BR', 'TUFF_CH', '']
# Read all CSV files in the directory
csv_files = all_files = glob.glob(os.path.join(directory , 'epd_ivscan_20*'))
# Concatenate all CSV files into a single dataframe
dataframes = {os.path.basename(file): pd.read_csv(file, names=columns) for file in csv_files}
df = pd.concat(dataframes.values(), ignore_index=True)

# drop the last column
df = df.drop(df.columns[-1], axis=1)
df.head()

# Print the names of the loaded files
# print(f"Loaded CSV files: {list(dataframes.keys())}")



/var/folders/n_/wmbdlzd17s7gcyvkql_0tn3c0000gq/T/ipykernel_1631/3343335863.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dataframes.values(), ignore_index=True)


,timestamp,address,ew,pp,tt,fee_channel,fee,VCAL0,VPED,VSLOPE,VSET,VCOMP,IMON,RMON,TUFF,TUFF_BR,TUFF_CH
0,1486554605,0x1100000032C0493A,-1.0,-2.0,-1.0,0.0,67.0,42.0,0.054,53.0,0.966,0.01,1.219,NaN,NaN,NaN,NaN
1,1486554605,0x1100000032C0493A,-1.0,-2.0,-2.0,1.0,67.0,42.0,0.054,53.0,0.966,0.01,1.207,NaN,NaN,NaN,NaN
2,1486554605,0x1100000032C0493A,-1.0,-2.0,-3.0,2.0,67.0,42.0,0.054,53.0,0.966,0.01,1.229,NaN,NaN,NaN,NaN
3,1486554605,0x1100000032C0493A,-1.0,-2.0,-4.0,3.0,67.0,42.0,0.054,53.0,0.966,0.01,1.217,NaN,NaN,NaN,NaN
4,1486554605,0x1100000032C0493A,-1.0,-2.0,-5.0,4.0,67.0,42.0,0.054,53.0,0.966,0.01,1.225,NaN,NaN,NaN,NaN


In [3]:
print(df.shape)
df = df.dropna(axis=0)
df = df[df['TUFF'] <= 2] # Why are there sometimes more than 2?  Who knows
print(df.shape)

(54951846, 17)
(52520448, 17)


In [9]:
# Turn the timestamp column into a datetime object
df['date'] = pd.to_datetime(df['timestamp'], unit='s')
df.sort_values(by='date', inplace=True)
# else{this._temperature[row[EW]][row[PP]][row[TT]]=(row[vcomp]/row[vslope]
							    #    +8.6).toFixed(1);}
df = df[df['VSLOPE'] != 0]
df['temperature'] =  df['VCOMP'] / df['VSLOPE'] + 8.6
df.tail()


,timestamp,address,ew,pp,tt,fee_channel,fee,VCAL0,VPED,VSLOPE,VSET,VCOMP,IMON,RMON,TUFF,TUFF_BR,TUFF_CH,date,temperature
8064820,1750690267,0x2900000116D0C870,0.0,9.0,9.0,4.0,70.0,67.0,-10.0,0.054,65.4,1.234,40.95,1.274,2.0,0.0,4.0,2025-06-23 14:51:07,31.451852
8064943,1750690267,0x4400000116CF5270,1.0,9.0,30.0,15.0,22.0,67.0,18.0,0.054,65.4,1.263,29.17,1.360,1.0,1.0,5.0,2025-06-23 14:51:07,31.988889
8064812,1750690267,0x2300000116E10C70,1.0,8.0,24.0,12.0,17.0,67.0,13.0,0.054,65.4,1.098,20.14,1.357,1.0,1.0,3.0,2025-06-23 14:51:07,28.933333
8064849,1750690267,0xAA00000116D6AC70,1.0,3.0,3.0,1.0,18.0,67.0,3.0,0.054,65.4,1.063,40.95,1.250,1.0,0.0,4.0,2025-06-23 14:51:07,28.285185
8064753,1750690267,0xBF00000116C36370,0.0,8.0,2.0,1.0,67.0,67.0,-1.0,0.054,65.4,1.102,40.95,1.296,2.0,0.0,3.0,2025-06-23 14:51:07,29.007407


In [11]:
# --- Load main DataFrame ---
# Make sure df is already loaded and 'date' is in datetime format
df['date'] = pd.to_datetime(df['date'])

# --- Load system/energy time ranges ---
log_df = pd.read_csv('STAR_runs_log.csv')

# Convert to datetime safely
log_df['start_date'] = pd.to_datetime(
    log_df[['Start_Year', 'Start_Month', 'Start_Day']].astype(str).agg('-'.join, axis=1),
    errors='coerce'
)
log_df['end_date'] = pd.to_datetime(
    log_df[['End_Year', 'End_Month', 'End_Day']].astype(str).agg('-'.join, axis=1),
    errors='coerce'
)

# Drop rows with invalid dates
log_df.dropna(subset=['start_date', 'end_date'], inplace=True)

# Define unique colors for systems
unique_systems = log_df['System'].unique()
color_map = plt.cm.get_cmap('tab10', len(unique_systems))
system_colors = {system: color_map(i) for i, system in enumerate(unique_systems)}


/var/folders/n_/wmbdlzd17s7gcyvkql_0tn3c0000gq/T/ipykernel_1631/2801546005.py:23: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  color_map = plt.cm.get_cmap('tab10', len(unique_systems))


In [ ]:
groups = df.groupby(['ew', 'pp', 'tt'])
groups = df.groupby(['TUFF', 'TUFF_BR', 'TUFF_CH'])
with PdfPages('epd_temperature_current_byrun.pdf') as pdf:
    for name, group in groups:
        tuff, tuff_br, tuff_ch = name
        fig, ax1 = plt.subplots(figsize=(12, 6))

        # Plot IMON (Dark Current) on LEFT Y-axis
        ax1.plot(group['date'], group['IMON'], '-', color='blue', label='Dark Current ($\mu$A)')
        ax1.set_ylabel('Dark Current ($\mu$A)', color='blue')
        ax1.set_ylim(0, 60)
        ax1.tick_params(axis='y', labelcolor='blue')

        # Plot Temperature on RIGHT Y-axis
        ax2 = ax1.twinx()
        ax2.plot(group['date'], group['temperature'], '-', color='red', label='Temperature (°C)')
        ax2.set_ylabel('Temperature (°C)', color='red')
        ax2.set_ylim(10, 50)
        ax2.tick_params(axis='y', labelcolor='red')

        # Title and layout
        plt.title(f'Temperature & Dark Current (TUFF={tuff}, Branch={tuff_br}, Channel={tuff_ch})')
        ax1.set_xlabel('Time')
        ax1.grid(True)
        fig.tight_layout()

        # Format the x-axis with monthly ticks
        ax1.xaxis.set_major_locator(MonthLocator(interval=4))
        ax1.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
        fig.autofmt_xdate()

        # Add background shading and System–Energy labels (positioned inside the plot)
        for _, row in log_df.iterrows():
            label = f"{row['System']} – {row['Energy']}"
            color = system_colors.get(row['System'], 'gray')
            start = row['start_date']
            end = row['end_date']

            # Shaded background
            ax1.axvspan(start, end, color=color, alpha=0.2)

            # Label placed at y=58 (just inside the top of the IMON axis)
            mid = start + (end - start) / 2
            ax1.text(mid, 58, label, rotation=90, va='top', ha='center', fontsize=9, color='black')

        pdf.savefig()
        plt.close()

<>:9: SyntaxWarning: invalid escape sequence '\m'
<>:10: SyntaxWarning: invalid escape sequence '\m'
<>:9: SyntaxWarning: invalid escape sequence '\m'
<>:10: SyntaxWarning: invalid escape sequence '\m'
/var/folders/n_/wmbdlzd17s7gcyvkql_0tn3c0000gq/T/ipykernel_1631/2274785130.py:9: SyntaxWarning: invalid escape sequence '\m'
  ax1.plot(group['date'], group['IMON'], '-', color='blue', label='Dark Current ($\mu$A)')
/var/folders/n_/wmbdlzd17s7gcyvkql_0tn3c0000gq/T/ipykernel_1631/2274785130.py:10: SyntaxWarning: invalid escape sequence '\m'
  ax1.set_ylabel('Dark Current ($\mu$A)', color='blue')


In [16]:
groups = df.groupby(['ew', 'pp', 'tt'])
groups = df.groupby(['TUFF', 'TUFF_BR', 'TUFF_CH'])
with PdfPages('epd_current_byrun.pdf') as pdf:
    for name, group in groups:
        tuff, tuff_br, tuff_ch = name
        fig, ax1 = plt.subplots(figsize=(12, 6))

        # Plot IMON (Dark Current) on LEFT Y-axis
        ax1.plot(group['date'], group['IMON'], '-', color='blue', label='Dark Current ($\mu$A)')
        ax1.set_ylabel('Dark Current ($\mu$A)', color='blue')
        ax1.set_ylim(0, 60)
        ax1.tick_params(axis='y', labelcolor='blue')

        # Plot Temperature on RIGHT Y-axis
       # ax2 = ax1.twinx()
       # ax2.plot(group['date'], group['temperature'], '-', color='red', label='Temperature (°C)')
       # ax2.set_ylabel('Temperature (°C)', color='red')
       # ax2.set_ylim(10, 50)
       # ax2.tick_params(axis='y', labelcolor='red')

        # Title and layout
        plt.title(f'Dark Current (TUFF={tuff}, Branch={tuff_br}, Channel={tuff_ch})')
        ax1.set_xlabel('Time')
        ax1.grid(True)
        fig.tight_layout()

        # Format the x-axis with monthly ticks
        ax1.xaxis.set_major_locator(MonthLocator(interval=4))
        ax1.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
        fig.autofmt_xdate()

        # Add background shading and System–Energy labels (positioned inside the plot)
        for _, row in log_df.iterrows():
            label = f"{row['System']} – {row['Energy']}"
            color = system_colors.get(row['System'], 'gray')
            start = row['start_date']
            end = row['end_date']

            # Shaded background
            ax1.axvspan(start, end, color=color, alpha=0.2)

            # Label placed at y=58 (just inside the top of the IMON axis)
            mid = start + (end - start) / 2
            ax1.text(mid, 58, label, rotation=90, va='top', ha='center', fontsize=9, color='black')

        pdf.savefig()
        plt.close()

<>:9: SyntaxWarning: invalid escape sequence '\m'
<>:10: SyntaxWarning: invalid escape sequence '\m'
<>:9: SyntaxWarning: invalid escape sequence '\m'
<>:10: SyntaxWarning: invalid escape sequence '\m'
/var/folders/n_/wmbdlzd17s7gcyvkql_0tn3c0000gq/T/ipykernel_1631/544326582.py:9: SyntaxWarning: invalid escape sequence '\m'
  ax1.plot(group['date'], group['IMON'], '-', color='blue', label='Dark Current ($\mu$A)')
/var/folders/n_/wmbdlzd17s7gcyvkql_0tn3c0000gq/T/ipykernel_1631/544326582.py:10: SyntaxWarning: invalid escape sequence '\m'
  ax1.set_ylabel('Dark Current ($\mu$A)', color='blue')


ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing